In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import xgboost as xgb

#from sklearn.grid_search import GridSearchCV
#from sklearn.metrics import log_loss
#from sklearn.ensemble import RandomForestClassifier
from IPython.display import display

from Util.data_utils import ReadCSV
from Util.Utils import FirstLook

In [ ]:
Path = "D:\\test\\Kaggle\\Util\\dataset\\Instacart"
#aisles, departments, order_products__prior, order_products__train, orders, products, sample_submission = \
#ReadCSV("aisles.csv", "departments.csv", "order_products__prior.csv", "order_products__train.csv", \
#"orders.csv", "products.csv", "sample_submission.csv", path=Path)
order_products__train = \
ReadCSV( "order_products__train.csv", path=Path)

In [2]:
#Final Number of products
orders_copy=pd.read_pickle('dataset_1.pickle')
orders_copy.head(5)

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,order_dayom,order_dom,dow_score,hod_score,dom_score,orders_score,product_count,weighted_product_count,num_products
0,2539329,1,prior,1,2,8,NaN,1.0,1.0,4.0,23.0,19.0,1.793103,5.0,8.965517,6.0
1,2398795,1,prior,2,3,7,15.0,16.0,16.0,5.0,22.0,24.0,2.879310,6.0,17.275862,6.0
2,473747,1,prior,3,3,12,21.0,37.0,7.0,5.0,19.0,25.0,3.844828,5.0,19.224138,6.0
3,2254736,1,prior,4,4,7,29.0,66.0,6.0,6.0,22.0,24.0,4.896552,5.0,24.482759,6.0
4,431534,1,prior,5,4,15,28.0,94.0,4.0,6.0,16.0,22.0,5.758621,8.0,46.068966,6.0


In [3]:
order_num_normalizer = np.mean(orders_copy['order_number'])
order_num_normalizer

17.154857979183785

In [4]:
#normalizing order score
orders_copy['orders_score'] = orders_copy['order_number']/order_num_normalizer + ((orders_copy['dow_score']+orders_copy['hod_score']+ \
                                                             orders_copy['dom_score'])/(6+23+29))

In [5]:
orders_copy['weighted_product_count'] = orders_copy.loc[:,'product_count']*orders_copy.loc[:,'orders_score']

orders_copy['num_products'] = pd.Series(np.zeros(orders_copy.shape[0]))

def Get_Number_of_Products(df):
    df.loc[:, 'num_products'] = np.sum(df.loc[:, 'weighted_product_count'])/np.sum(df.loc[:, 'orders_score'])
    return df

orders_copy = orders_copy.groupby(['user_id']).apply(Get_Number_of_Products)
orders_copy['num_products']=np.ceil(orders_copy['num_products'])
orders_copy.head(5)

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,order_dayom,order_dom,dow_score,hod_score,dom_score,orders_score,product_count,weighted_product_count,num_products
0,2539329,1,prior,1,2,8,NaN,1.0,1.0,4.0,23.0,19.0,0.851396,5.0,4.256980,6.0
1,2398795,1,prior,2,3,7,15.0,16.0,16.0,5.0,22.0,24.0,0.995895,6.0,5.975372,6.0
2,473747,1,prior,3,3,12,21.0,37.0,7.0,5.0,19.0,25.0,1.019705,5.0,5.098526,6.0
3,2254736,1,prior,4,4,7,29.0,66.0,6.0,6.0,22.0,24.0,1.129722,5.0,5.648609,6.0
4,431534,1,prior,5,4,15,28.0,94.0,4.0,6.0,16.0,22.0,1.050083,8.0,8.400667,6.0


In [6]:
orders_copy.to_pickle('scored_orders.pickle')

In [7]:
del orders_copy

In [9]:
merged_dataset=pd.read_csv('merged_dataset.csv', index_col=0)
merged_dataset.head(3)

,order_id,product_id,add_to_cart_order,reordered,department_id,orders_score,user_id,products_score
0,2,33120,1,1,16,3.810345,202279,3.810345
1,2,28985,2,1,4,3.810345,202279,3.810345
2,2,9327,3,0,13,1.905172,202279,1.905172


In [12]:
merged_dataset.head(3)

,order_id,product_id,add_to_cart_order,reordered,department_id,user_id
0,2,33120,1,1,16,202279
1,2,28985,2,1,4,202279
2,2,9327,3,0,13,202279


In [13]:
orders_copy=pd.read_pickle('scored_orders.pickle')

In [14]:
merged_dataset=pd.merge(merged_dataset, orders_copy.loc[:,['order_id', 'orders_score']], on='order_id', how='left')
merged_dataset.head(5)

,order_id,product_id,add_to_cart_order,reordered,department_id,user_id,orders_score
0,2,33120,1,1,16,202279,0.985222
1,2,28985,2,1,4,202279,0.985222
2,2,9327,3,0,13,202279,0.985222
3,2,45918,4,1,13,202279,0.985222
4,2,30035,5,0,13,202279,0.985222


In [15]:
del orders_copy

In [17]:
merged_dataset.to_pickle('product_score_raw.pickle')

In [18]:
const=0.8
merged_dataset.loc[:, 'products_score']=merged_dataset.loc[:,'orders_score']*(1-(0.8-0.8*merged_dataset.loc[:,'reordered']))
merged_dataset.head(3)

,order_id,product_id,add_to_cart_order,reordered,department_id,user_id,orders_score,products_score
0,2,33120,1,1,16,202279,0.985222,0.985222
1,2,28985,2,1,4,202279,0.985222,0.985222
2,2,9327,3,0,13,202279,0.985222,0.197044


In [20]:
product_score=merged_dataset.groupby(['user_id','product_id']).agg({'products_score':np.sum}).reset_index()
product_score.head(4)

,user_id,product_id,products_score
0,1,196,10.266351
1,1,10258,9.299356
2,1,10326,0.210017
3,1,12427,10.266351


In [21]:
product_score.to_pickle('product_score_dataset.pickle')

Now lets find number of products for each training data

In [37]:
#this
#trainset number of products vs number of products predicted : products_count = actual count, num_products = predicted
train_product_count=pd.read_pickle('train_product_count.pickle')
del train_product_count['num_products']
train_product_count.head(4)

,order_id,products_count
0,1,8
1,36,8
2,38,9
3,96,7


In [23]:
#this
orders_copy=pd.read_pickle('scored_orders.pickle')
orders_copy.head(3)

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,order_dayom,order_dom,dow_score,hod_score,dom_score,orders_score,product_count,weighted_product_count,num_products
0,2539329,1,prior,1,2,8,NaN,1.0,1.0,4.0,23.0,19.0,0.851396,5.0,4.256980,6.0
1,2398795,1,prior,2,3,7,15.0,16.0,16.0,5.0,22.0,24.0,0.995895,6.0,5.975372,6.0
2,473747,1,prior,3,3,12,21.0,37.0,7.0,5.0,19.0,25.0,1.019705,5.0,5.098526,6.0


In [38]:
#this
train_product_count=pd.merge(train_product_count, orders_copy.loc[:,['user_id','order_id','num_products']], on='order_id', how='left')
train_product_count.head(5)

,order_id,products_count,user_id,num_products
0,1,8,112108,7.0
1,36,8,79431,9.0
2,38,9,42756,8.0
3,96,7,17227,7.0
4,98,49,56463,31.0


In [39]:
#this
rmse = np.sqrt(np.sum((train_product_count.loc[:,'products_count']-train_product_count.loc[:,'num_products'])**2)/train_product_count.shape[0])
display(rmse)

5.7737427196574798

In [43]:
train_data=orders_copy.loc[orders_copy.loc[:,'eval_set']=='test',['order_id', 'user_id']]
train_data.head(4)

,order_id,user_id
38,2774568,3
44,329954,4
53,1528013,6
96,1376945,11


Getting product count for prior data

In [46]:
Path = "D:\\test\\Kaggle\\Util\\dataset\\Instacart"
#aisles, departments, order_products__prior, order_products__train, orders, products, sample_submission = \
#ReadCSV("aisles.csv", "departments.csv", "order_products__prior.csv", "order_products__train.csv", \
#"orders.csv", "products.csv", "sample_submission.csv", path=Path)
order_products_prior = ReadCSV( "order_products__prior.csv", path=Path)
order_products_prior=order_products_prior[0]
order_products_prior.head(4)

D:\test\Kaggle\Util\dataset\Instacart\order_products__prior.csv


,order_id,product_id,add_to_cart_order,reordered
0,2,33120,1,1
1,2,28985,2,1
2,2,9327,3,0
3,2,45918,4,1


Getting product count for each order

In [47]:

order_products_prior['products_count']=1
order_products_prior = order_products_prior.groupby(['order_id']).agg({'products_count':np.sum}).reset_index()
order_products_prior.head(3)

,order_id,products_count
0,2,9
1,3,8
2,4,13


In [49]:
#number of products for test orders and user_ids
test_data=orders_copy.loc[orders_copy.loc[:,'eval_set']=='test',['order_id', 'user_id', 'num_products']]
test_data.head(4)

,order_id,user_id,num_products
38,2774568,3,7.0
44,329954,4,4.0
53,1528013,6,5.0
96,1376945,11,13.0


Combining number of products with product score for each order_id in test data

In [56]:
product_score_right=pd.merge(product_score, test_data.loc[:,['user_id','order_id']], on='user_id', how='right')
product_score_right.head(5)

,user_id,product_id,products_score,num_products,order_id
0,3,248,0.150903,7.0,2774568
1,3,1005,0.282102,7.0,2774568
2,3,1819,2.487195,7.0,2774568
3,3,7503,0.203941,7.0,2774568
4,3,8021,0.150903,7.0,2774568


coming up with the output resultand products

In [61]:
def Output(df):
    num_prod=int(df['num_products'].iloc[0])
    tmp=df.nlargest(num_prod, 'products_score')
    return tmp.loc[:,['order_id', 'product_id']]

ot=product_score_right.groupby('order_id').apply(Output)
ot.head(15)

order_id  product_id
order_id                             
17       858082        17       13107
         858087        17       21463
         858092        17       38777
         858088        17       21709
         858097        17       47766
         858088        17       21709
         858097        17       47766
         858090        17       26429
         858093        17       39275
         858090        17       26429
         858093        17       39275
34       814891        34       39180
         814910        34       47029
         814893        34       39475
         814912        34       47792

In [69]:
ot.product_id=ot.product_id.astype(str)

In [70]:
ot_final=ot.groupby('order_id')['product_id'].apply(' '.join).reset_index()
ot_final.head()

,order_id,product_id
0,17,13107 21463 38777 21709 47766 21709 47766 2642...
1,34,39180 47029 39475 47792 47766 43504 2596 16083...
2,137,23794 41787 23794 41787 38689 24852 5134 2326 ...
3,182,9337 39275 13629 27104 5479 47209 47672 33000 ...
4,257,49235 27104 24852 29837 45013 27966 13870 4605...


In [73]:
ot_final.columns=['order_id', 'products']
ot_final.to_csv('submission.csv', index=False)